In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
import random

In [5]:
def factorizer(arr):
    d={}
    c = 0
    for i in arr:
        if i not in d:
            d[i] = c
            c+=1
        else:
            pass
    return d, [d[i] for i in arr]

In [6]:
data = pd.read_csv('All-seasons.csv')[:10000]
data['Line'] = [line.strip() for line in data['Line']]
factors_dict, data['CharFact'] = factorizer(data['Character'])
data.head()

,Season,Episode,Character,Line,CharFact
0,10,1,Stan,"You guys, you guys! Chef is going away.",0
1,10,1,Kyle,Going away? For how long?,1
2,10,1,Stan,Forever.,0
3,10,1,Chef,I'm sorry boys.,2
4,10,1,Stan,"Chef said he's been bored, so he joining a gro...",0


In [7]:
data['Character'].describe()

count       10000
unique        924
top       Cartman
freq         1447
Name: Character, dtype: object

In [8]:
def baseline(data):
    return [random.randint(0, 923) for i in range(len(data))]

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectrzr = TfidfVectorizer()
matrix = vectrzr.fit_transform(data['Line'])

Я решила не лемматизировать слова, потому что подумала, что есть возможность, что это несколько сотрёт разницу между репликами различных песонажей. Кроме того, я не использую стоп-слова, потому что у меня тут tf-idf, который и так уже умеет работать со словами разной частотности. Обсценную лексику в стоплист включать, конечно, тоже не надо -- это такая же важная информация о тексте, как и всё остальное.

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(matrix, data['CharFact'], test_size=0.2)

In [13]:
from sklearn.metrics import classification_report

In [14]:
print(classification_report(y_train, baseline(y_train)))

             precision    recall  f1-score   support

          0       0.14      0.00      0.00       691
          1       0.00      0.00      0.00       788
          2       0.00      0.00      0.00        39
          3       0.00      0.00      0.00       116
          4       0.11      0.00      0.00      1164
          5       0.00      0.00      0.00        10
          6       0.00      0.00      0.00        26
          7       0.00      0.00      0.00       107
          8       0.00      0.00      0.00        53
          9       0.00      0.00      0.00         2
         10       0.00      0.00      0.00         1
         11       0.10      0.00      0.01       355
         12       0.00      0.00      0.00         0
         13       0.00      0.00      0.00         1
         14       0.00      0.00      0.00        65
         15       0.00      0.00      0.00         4
         16       0.00      0.00      0.00        50
         17       0.00      0.00      0.00   

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:
from sklearn.metrics import accuracy_score

In [16]:
accuracy_score(y_train, baseline(y_train))

0.00087500000000000002

In [17]:
len(y_train)

8000

Как видно, baseline вообще практически ничего не угадывает, оно и неудивительно: у нас 924 класс и 8000 сэмплов.

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [19]:
parameters = {'C': [1, 10, 100, 1000], 'fit_intercept': [False, True], 'solver': ['newton-cg', 'lbfgs'], 'multi_class': ['ovr', 'multinomial']}

In [ ]:
logit2 = LogisticRegression()
logit2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
logit = LogisticRegression()
clf_logit = GridSearchCV(logit, parameters, verbose=2)
clf_logit.fit(X_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[CV] fit_intercept=False, multi_class=ovr, solver=newton-cg, C=1 .....
[CV]  fit_intercept=False, multi_class=ovr, solver=newton-cg, C=1, total=  35.5s
[CV] fit_intercept=False, multi_class=ovr, solver=newton-cg, C=1 .....


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.6s remaining:    0.0s


[CV]  fit_intercept=False, multi_class=ovr, solver=newton-cg, C=1, total= 1.2min
[CV] fit_intercept=False, multi_class=ovr, solver=newton-cg, C=1 .....
[CV]  fit_intercept=False, multi_class=ovr, solver=newton-cg, C=1, total= 1.1min
[CV] fit_intercept=False, multi_class=ovr, solver=lbfgs, C=1 .........
[CV]  fit_intercept=False, multi_class=ovr, solver=lbfgs, C=1, total=  22.3s
[CV] fit_intercept=False, multi_class=ovr, solver=lbfgs, C=1 .........
[CV]  fit_intercept=False, multi_class=ovr, solver=lbfgs, C=1, total= 1.1min
[CV] fit_intercept=False, multi_class=ovr, solver=lbfgs, C=1 .........
[CV]  fit_intercept=False, multi_class=ovr, solver=lbfgs, C=1, total=  39.2s
[CV] fit_intercept=False, multi_class=multinomial, solver=newton-cg, C=1 
[CV]  fit_intercept=False, multi_class=multinomial, solver=newton-cg, C=1, total=  59.8s
[CV] fit_intercept=False, multi_class=multinomial, solver=newton-cg, C=1 
[CV]  fit_intercept=False, multi_class=multinomial, solver=newton-cg, C=1, total= 1.5m

[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed: 449.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'fit_intercept': [False, True], 'solver': ['newton-cg', 'lbfgs'], 'multi_class': ['ovr', 'multinomial'], 'C': [1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=2)

In [28]:
print(classification_report(logit.predict(X_train), y_train))

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


             precision    recall  f1-score   support

          0       0.79      0.63      0.70       898
          1       0.81      0.67      0.73       945
          2       0.84      0.94      0.89        33
          3       0.80      0.86      0.83       107
          4       0.91      0.75      0.82      1381
          5       1.00      1.00      1.00        10
          6       0.80      0.95      0.87        21
          7       0.80      0.93      0.86        81
          8       0.92      0.96      0.94        46
          9       0.50      1.00      0.67         1
         10       0.00      0.00      0.00         0
         11       0.82      0.86      0.84       337
         12       0.00      0.00      0.00         0
         13       1.00      1.00      1.00         1
         14       0.73      0.82      0.77        55
         15       0.00      0.00      0.00         0
         16       0.55      0.88      0.67        32
         17       0.33      0.67      0.44   

In [25]:
clf_logit.best_params_

{'C': 10,
 'fit_intercept': True,
 'multi_class': 'multinomial',
 'solver': 'lbfgs'}

In [ ]:
logit = LogisticRegression(C=10, fit_intercept=True, multi_class='multinomial', solver='lbfgs')
logit.fit(X_train, y_train)

In [20]:
logit.intercept_

array([-3.21413277, -3.08389147, -6.05260412, -5.71677433, -3.13970807,
       -7.40043724, -6.56302597, -5.80107971, -6.62804453, -9.24588519,
       -9.26774627, -4.18596292, -9.76537907, -9.25078542, -5.50322148,
       -8.64920102, -6.07882701, -6.91038975, -7.90716253, -3.83065132,
       -9.68118039, -7.193088  , -8.56021343, -9.10043989, -6.63408496,
       -8.80846461, -9.37634166, -6.18077786, -7.44716646, -7.2201635 ,
       -8.62599078, -8.86775034, -9.10811572, -9.1776604 , -8.71657235,
       -7.78184291, -8.66452473, -8.49462419, -8.71857673, -6.46412139,
       -7.60551495, -5.81891871, -6.59402736, -5.78118884, -5.88849951,
       -6.62309943, -6.86199735, -7.04191132, -7.88598681, -9.47854607,
       -9.43893086, -9.41911814, -9.37163262, -9.37163262, -9.27752645,
       -7.32876094, -6.27004028, -9.21415365, -7.91870407, -8.51732904,
       -9.35841674, -8.14346656, -6.85763056, -7.98803163, -8.55046509,
       -9.47328079, -9.26978098, -5.93964349, -6.60011262, -6.41

Это, кажется, свободные коэффициенты для каждого из классов, которые решают, относится ли объект к классу, когда все остальные признаки равны нулю.

In [27]:
logit.coef_

array([[ -4.94738808e-02,   1.61274519e+00,  -5.66983974e-01, ...,
         -7.12491374e-01,  -2.60440994e-01,   3.68755968e+00],
       [ -7.49848790e-02,  -9.40454168e-02,  -1.06641138e+00, ...,
         -8.87603208e-01,   2.52818651e+00,  -2.40640561e-01],
       [ -5.25887498e-03,  -6.16870698e-03,  -2.83307013e-02, ...,
         -1.10347307e-01,  -1.09488923e-02,  -2.14583597e-02],
       ..., 
       [ -1.63196899e-03,  -1.92735892e-03,  -1.64305305e-02, ...,
         -2.18486825e-02,  -3.03915360e-03,  -5.25494618e-03],
       [ -9.51356515e-04,  -1.12334500e-03,  -7.31308534e-03, ...,
         -9.46378830e-03,  -2.06785916e-03,  -3.42290565e-03],
       [ -4.53297693e-04,  -5.78133167e-04,  -3.72519636e-03, ...,
         -5.00445441e-03,  -9.55823725e-04,  -1.42724352e-03]])

Это матрица коэффициентов регрессии, тут для каждого класса показано, как каждая фича на него влияет. Если коэф. положительный, то вероятность отнесения элемента с такой фичей к этому классу увеличивается, если отрицательный, то уменьшается. Присваивается тот класс, вероятность которого оказывается в итоге выше всего.

In [17]:
from sklearn.ensemble import RandomForestClassifier
forest_params = {'max_features':[.25, .3, .5, 'sqrt', 'log2', None], 'max_depth':[10, 100, 1000, 10000, None], 'bootstrap':[True, False]}

In [20]:
clf_forest = GridSearchCV(RandomForestClassifier(), forest_params, verbose=10)
clf_forest.fit(X_train, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[CV] bootstrap=True, max_depth=10, max_features=0.25 .................
[CV]  bootstrap=True, max_depth=10, max_features=0.25, score=0.150196, total=   2.0s
[CV] bootstrap=True, max_depth=10, max_features=0.25 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV]  bootstrap=True, max_depth=10, max_features=0.25, score=0.175540, total=   3.1s
[CV] bootstrap=True, max_depth=10, max_features=0.25 .................


...

[CV]  bootstrap=True, max_depth=10, max_features=0.5, score=0.202381, total=   6.2s
[CV] bootstrap=True, max_depth=10, max_features=sqrt .................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   38.9s remaining:    0.0s


[CV]  bootstrap=True, max_depth=10, max_features=sqrt, score=0.138743, total=   0.4s
[CV] bootstrap=True, max_depth=10, max_features=sqrt .................
[CV]  bootstrap=True, max_depth=10, max_features=sqrt, score=0.161265, total=   0.5s
[CV] bootstrap=True, max_depth=10, max_features=sqrt .................
[CV]  bootstrap=True, max_depth=10, max_features=sqrt, score=0.180272, total=   0.5s
[CV] bootstrap=True, max_depth=10, max_features=log2 .................
[CV]  bootstrap=True, max_depth=10, max_features=log2, score=0.127291, total=   0.3s
[CV] bootstrap=True, max_depth=10, max_features=log2 .................
[CV]  bootstrap=True, max_depth=10, max_features=log2, score=0.150077, total=   0.4s
[CV] bootstrap=True, max_depth=10, max_features=log2 .................
[CV]  bootstrap=True, max_depth=10, max_features=log2, score=0.162840, total=   0.4s
[CV] bootstrap=True, max_depth=10, max_features=None .................
[CV]  bootstrap=True, max_depth=10, max_features=None, score=0.1

[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed: 128.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [10, 100, 1000, 10000, None], 'bootstrap': [True, False], 'max_features': [0.25, 0.3, 0.5, 'sqrt', 'log2', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [21]:
clf_forest.best_params_

{'bootstrap': True, 'max_depth': 1000, 'max_features': 0.25}

In [29]:
print(classification_report(clf_forest.predict(X_train), y_train))

             precision    recall  f1-score   support

          0       0.90      0.88      0.89       726
          1       0.93      0.78      0.85       933
          2       0.95      0.78      0.85        45
          3       0.92      0.97      0.95       109
          4       0.96      0.93      0.94      1182
          5       1.00      1.00      1.00        10
          6       0.96      1.00      0.98        24
          7       0.96      0.97      0.96        93
          8       0.94      0.96      0.95        47
          9       1.00      1.00      1.00         2
         10       1.00      1.00      1.00         1
         11       0.92      0.98      0.95       334
         12       1.00      1.00      1.00         1
         13       1.00      1.00      1.00         1
         14       0.84      0.90      0.87        58
         15       1.00      1.00      1.00         1
         16       0.80      1.00      0.89        41
         17       0.67      0.67      0.67   

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [30]:
from sklearn.naive_bayes import MultinomialNB
bayes_params = {'alpha':[0.0, .1, .25, .3, .5, .75, .9, 1.0], 'fit_prior':[True, False]}

In [31]:
clf_bayes = MultinomialNB(alpha=0.00000001, fit_prior=False)
clf_bayes.fit(X_train, y_train)

MultinomialNB(alpha=1e-08, class_prior=None, fit_prior=False)

GridSearch применить не удалось, параметры подбирались вручную.

In [32]:
print(classification_report(clf_bayes.predict(X_train), y_train))

             precision    recall  f1-score   support

          0       0.54      0.85      0.66       456
          1       0.52      0.88      0.66       464
          2       0.62      0.79      0.70        29
          3       0.75      0.78      0.76       110
          4       0.68      0.98      0.81       804
          5       1.00      0.83      0.91        12
          6       0.80      0.69      0.74        29
          7       0.76      0.78      0.77        91
          8       0.88      0.76      0.82        55
          9       1.00      1.00      1.00         2
         10       1.00      0.14      0.25         7
         11       0.70      0.91      0.79       276
         12       1.00      1.00      1.00         1
         13       1.00      1.00      1.00         1
         14       0.61      0.61      0.61        62
         15       1.00      0.50      0.67         2
         16       0.61      0.57      0.59        54
         17       0.33      0.57      0.42   

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Как видно, лучшей моделью у нас оказался лес с максимальной глубиной дерева 1000, максимальным количеством фич для разбиения выборки на 2 части -- 25% от общего кол-ва фич, и с бутстрэппингом (я так и не поняла, что это).

***Финальная битва на тестовых данных***

In [33]:
print(classification_report(clf_logit.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.39      0.17      0.24       385
          1       0.39      0.17      0.23       442
          2       0.44      0.67      0.53         6
          3       0.32      0.36      0.34        25
          4       0.59      0.28      0.38       615
          5       0.00      0.00      0.00         0
          6       0.00      0.00      0.00         0
          7       0.03      0.08      0.04        13
          8       0.38      0.50      0.43        10
          9       0.00      0.00      0.00         0
         11       0.27      0.19      0.22       128
         14       0.20      0.27      0.23        15
         15       0.00      0.00      0.00         0
         16       0.00      0.00      0.00         8
         17       0.43      1.00      0.60         3
         18       0.00      0.00      0.00         0
         19       0.31      0.24      0.27       100
         21       0.00      0.00      0.00   

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [34]:
print(classification_report(clf_forest.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.32      0.14      0.19       390
          1       0.35      0.16      0.22       415
          2       0.44      0.27      0.33        15
          3       0.25      0.26      0.25        27
          4       0.55      0.28      0.37       581
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         1
          7       0.06      0.15      0.09        13
          8       0.69      0.56      0.62        16
          9       0.00      0.00      0.00         0
         11       0.21      0.21      0.21        91
         14       0.05      0.11      0.07         9
         15       0.00      0.00      0.00         0
         16       0.00      0.00      0.00         2
         17       0.57      0.67      0.62         6
         18       0.00      0.00      0.00         0
         19       0.22      0.25      0.23        69
         21       0.00      0.00      0.00   

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [35]:
print(classification_report(clf_bayes.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.20      0.20      0.20       173
          1       0.21      0.19      0.20       208
          2       0.22      0.40      0.29         5
          3       0.25      0.15      0.18        48
          4       0.45      0.32      0.37       419
          5       0.00      0.00      0.00         0
          6       0.00      0.00      0.00         2
          7       0.03      0.04      0.03        28
          8       0.15      0.12      0.14        16
          9       0.00      0.00      0.00         1
         10       0.00      0.00      0.00         2
         11       0.31      0.24      0.27       117
         14       0.00      0.00      0.00         8
         15       0.00      0.00      0.00         0
         16       0.09      0.06      0.07        16
         17       0.00      0.00      0.00         0
         18       0.00      0.00      0.00         0
         19       0.17      0.24      0.20   

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


На тестовой выборке логистическая регрессия справилась лучше всего, но вообще результаты не очень впечатляют. Хотя вообще это не очень удивительно, если учесть, что у нас очень много персонажей (и большинство -- второстепенные), и для каждого объекта выборки надо угадывать, какому персонажу принадлежит реплика, по очень маленькому количеству слов.

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, verbose=1)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
title = "Learning Curve (Logistic Regression)"
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
estimator = logit
plot_learning_curve(estimator, title, matrix, data['CharFact'], ylim=(0.7, 1.01), cv=cv, n_jobs=4)